In [7]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sys;
import pprint

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords;
import nltk;

from gensim.models import ldamodel
from gensim.models import nmf
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import gensim

import sklearn;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;

import pickle;

In [2]:
data = pd.read_csv('/Users/kdwoo/Documents/Jupyter/2019_CAU_NLP/abcnews-date-text.csv', error_bad_lines=False);

In [3]:
stop_words = stopwords.words('english')

In [4]:
data['publish_date'] = pd.to_datetime(data['publish_date'].astype(str), format = '%Y%m%d')
data['publish_date'] = pd.DatetimeIndex(data['publish_date']).year

In [5]:
data_text = data[['headline_text']];

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['headline_text']))

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [11]:
data_words_nostops = pd.Series(data_words_nostops)
data_lemmatized = data_words_nostops.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [12]:
#data_words_nostops

In [13]:
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])  # 불용어처리

#  빈도수 높은 키워드 처리
stop_words.extend([ 'from', 'subject', 're', 'edu', 'use', 'home', 'hunter', 'help', 'time', 'national', 'build', 'end', 'bid', 'cup', 'un', 'come', 'join', 'across','Italy', 'grind', 'asian', 'sa', 'miss', 'one', 'use', 'three', 'Darwin', 'vic', 'number', 'may', 'start', 'law', 'way', 'communities', 'order', 'check', 'major', 'india', 'focus', 'form', 'journalist', 'milk', 'nz', 'rank', 'cook', 'egypt', 'New', 'year', 'force', 'fail', 'dead', 'was', 'fruit', 'philippines','nick'])
stop_words.extend(['fire', 'new', 'world', 'boat', 'turn', 'around', 'well', 'Find', 'two', 'adelaide', 'first', 'make', 'body', 'probe', 'outback', 'baby', 'David', 'street', 'mass', 'hotel', 'say', 'open', 'go', 'welcome', 'announce', 'level', 'allow', 'highest','queensland', 'kill', 'crash', 'road', 'record', 'nt', 'hit', 'plane', 'toll', 'suspend', 'peninsula', 'afghan', 'recovery','man', 'perth', 'flood', 'people', 'still', 'supply', 'siege', 'spark', 'summer', 'Michael', 'ops', 'large', 'flash', 'view', 'attack', 'back', 'mine', 'deal', 'fan', 'celebrate', 'target', 'hill', 'party', 'reveal', 'terrorism', 'video', 'pressure', 'remember', 'korea', 'indian', 'millions', 'drill', 'country', 'hour', 'podcast', 'leaders', 'thursday', 'abbott', 'tony',  'shorten', 'sach', 'day', 'years', 'show', 'teen', 'heat', 'issue', 'free', 'australias', 'asbestos', 'compete','South', 'china', 'talk', 'appeal', 'labor', 'plant', 'peter', 'allegedly', 'begin', 'try', 'ice', 'native', 'alcohol', 'Australia', 'league', 'live', 'launch',  'benefit', 'update', 'stream', 'cabinet', 'document', 'bob','Test', 'drug', 'brisbane', 'british', 'double', 'ebola', 'Wa', 'research', 'expansion', 'ready', 'old', 'release', 'paper', 'see'])

# 그 아래에서  빈도수 높은 키워드 처리
stop_words.extend(['call',  'queensland', 'melbourne', 'perth', 'thousands', 'alert', 'reveal', 'spark', 'amid', 'australian',  'brisbane', 'western', 'high', 'fan', 'prepare', 'british', 'battle', 'beach', 'wa', 'take',  'box', 'could',  'search', 'black', 'michael', 'week','man', 'day' ,'country', 'new', 'old', 'test',  'force', 'release', 'miss','say', 'south', 'was','fire', 'victoria', 'build','australia', 'court','find', 'fall','mine', 'darwin', 'break', 'record', 'david', 'reflect', 'remember','adelaide', 'show'])

#  빈도수 높은 키워드 처리
stop_words.extend(['from', 'subject', 're', 'edu', 'use',' court', 'home',' council', 'hunter', 'help', 'time', 'injure', 'national', 'build', 'end', 'bid', 'cup', 'un', 'come', 'security', 'volunteer', 'ship', 'crew', 'crowd', 'join', 'helicopter', 'across', 'museum', 'Italy', 'grind', 'asian', 'sa', 'miss', 'one', 'die', 'use', 'three', 'Darwin', 'vic', 'number', 'may', 'start', 'law', 'way', 'communities', 'order', 'check', 'major', 'india', 'focus', 'form', 'journalist', 'milk', 'nz', 'rank', 'cook', 'egypt', 'New', 'year', 'force', 'fail', 'dead', 'was', 'farmer', 'fruit', 'philippines', 'injury', 'nick'])
stop_words.extend(['fire', 'new', 'hobart', 'rural', 'world', 'boat', 'turn', 'flight', 'around', 'well', 'Find', 'two', 'adelaide', 'murder', 'first', 'make', 'body', 'probe', 'outback', 'tourism', 'baby', 'David', 'street', 'mass', 'hotel', 'Police', 'say', 'open', 'dog', 'go', 'welcome', 'president', 'announce', 'level', 'allow', 'highest','queensland', 'kill', 'crash', 'road', 'record', 'nt', 'hit', 'plane', 'toll', 'suspend', 'peninsula', 'afghan', 'recovery','man', 'perth', 'flood', 'people', 'prison', 'still', 'supply', 'siege', 'spark', 'summer', 'Michael', 'ops', 'large', 'flash', 'view', 'attack', 'back', 'mine', 'deal', 'fan', 'celebrate', 'target', 'hill', 'party', 'reveal', 'terrorism', 'video', 'pressure', 'remember', 'korea', 'indian', 'millions', 'drill', 'country', 'hour', 'podcast', 'leaders', 'thursday', 'abbott', 'tony', 'policy', 'agricultural', 'shorten', 'sach', 'day', 'years', 'show', 'teen', 'heat', 'sport', 'issue', 'free', 'australias', 'asbestos', 'compete','South', 'china', 'talk', 'appeal', 'labor', 'plant', 'peter', 'allegedly', 'begin', 'try', 'ice', 'native', 'alcohol', 'Australia', 'league', 'live', 'launch', 'campaign', 'benefit', 'update', 'stream', 'cabinet', 'document', 'bob','Test', 'drug', 'brisbane', 'international', 'british', 'double', 'treat', 'patient', 'ebola', 'Wa', 'bushfire', 'research', 'expansion', 'ready', 'old', 'release', 'paper', 'see'])

# 그 아래에서  빈도수 높은 키워드 처리
stop_words.extend(['call',  'queensland', 'melbourne', 'perth', 'thousands', 'alert', 'reveal', 'spark', 'amid', 'illegal', 'australian', 'price',  'brisbane', 'western', 'high', 'fan', 'prepare', 'british', 'battle', 'beach', 'wa', 'take',  'box', 'could',  'search', 'black', 'michael', 'week','man', 'day' ,'country', 'new', 'old', 'police', 'test',  'force', 'release', 'hobart', 'council', 'die', 'miss','say', 'south', 'was','fire', 'victoria', 'build','australia', 'court','find', 'fall','mine','attack', 'darwin', 'break', 'record', 'david', 'reflect', 'remember','adelaide', 'show'])
stop_words.extend(['set','cut','put','us','january','febrary','april','may','june','july', 'august', 'november', 'september', 'october', 'december', 'july', 'march', 'wednesday', 'february','monday','tuesday','wednesday','thursday','friday','sunday','get','act','sydney','iraq'])
#stop_words

# 2019-06-13
stop_words.extend(['govt', 'plan', 'continue', 'charge', 'face', 'five', 'qld'])

In [14]:
data_lemmatized = remove_stopwords(data_lemmatized)

In [15]:
temp = data
temp['lemmatize'] = data_lemmatized
for i in range(15):
    globals()['trend{}'.format(i+2003)] = temp.loc[temp.publish_date == i+2003]

In [16]:
trend_list = [trend2003, trend2004, trend2005, trend2006, trend2007, trend2008, trend2009, trend2010,
              trend2011, trend2012, trend2013, trend2014, trend2015, trend2016, trend2017]
#print(trend_list)

In [17]:
#pickle.dump(data_text, open('2003_data_text.dat', 'wb'))

In [18]:
#train_headlines = [value[0] for value in data_text.iloc[0:].values];
#print(train_headlines[0]);

In [36]:
num_topics = 20;
lemmatized = list(trend2006['lemmatize']) # 확인 년도

In [37]:
id2word = corpora.Dictionary(lemmatized)
texts = lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [38]:
#train_headlines_sentences = [' '.join(text) for text in train_headlines]
#print(train_headlines_sentences[0]);

In [39]:
vectorizer = CountVectorizer(analyzer='word', max_features=len(lemmatized));
x_counts = vectorizer.fit_transform(' '.join(text) for text in list(trend2006['lemmatize']));

In [40]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [41]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [42]:
nmf_model = gensim.models.nmf.Nmf(corpus,
                                   num_topics,
                                   id2word)

In [43]:
pprint.pprint(nmf_model.print_topics())

[(0,
  '0.102*"pm" + 0.046*"closer" + 0.033*"defend" + 0.016*"return" + '
  '0.016*"cyclone" + 0.013*"case" + 0.013*"lose" + 0.012*"bali" + 0.011*"pay" '
  '+ 0.010*"workers"'),
 (1,
  '0.068*"consider" + 0.059*"residents" + 0.031*"rescue" + 0.026*"game" + '
  '0.024*"nsw" + 0.017*"group" + 0.014*"rain" + 0.013*"safety" + 0.012*"team" '
  '+ 0.008*"move"'),
 (2,
  '0.121*"reject" + 0.047*"fear" + 0.028*"group" + 0.023*"claim" + 0.019*"car" '
  '+ 0.015*"work" + 0.010*"indigenous" + 0.009*"bomb" + 0.009*"question" + '
  '0.009*"laws"'),
 (3,
  '0.156*"report" + 0.034*"mp" + 0.019*"urge" + 0.018*"tax" + 0.018*"support" '
  '+ 0.017*"offer" + 0.011*"final" + 0.010*"play" + 0.008*"bird" + '
  '0.007*"flu"'),
 (4,
  '0.187*"health" + 0.099*"service" + 0.043*"mental" + 0.034*"urge" + '
  '0.018*"indigenous" + 0.009*"opposition" + 0.007*"ama" + 0.006*"push" + '
  '0.006*"action" + 0.006*"need"'),
 (5,
  '0.225*"seek" + 0.026*"opposition" + 0.026*"accuse" + 0.009*"support" + '
  '0.009*"group"

In [35]:
pickle.dump(nmf_model, open('2004_20topic_data_text.dat', 'wb'))

In [420]:
with open('/Users/kdwoo/Documents/Jupyter/2019_CAU_NLP/nmf_data/nmf_20_data/2003_20topic_data_text.dat', 'rb') as f:
    data_result = pickle.load(f)

In [422]:
pprint.pprint(data_result.print_topics())

[(0,
  '0.175*"support" + 0.019*"seek" + 0.015*"big" + 0.013*"group" + '
  '0.010*"offer" + 0.009*"strike" + 0.009*"centre" + 0.009*"community" + '
  '0.007*"business" + 0.007*"school"'),
 (1,
  '0.240*"claim" + 0.045*"reject" + 0.009*"title" + 0.007*"abuse" + '
  '0.005*"investigate" + 0.005*"victory" + 0.004*"sars" + 0.004*"hear" + '
  '0.004*"group" + 0.004*"opposition"'),
 (2,
  '0.160*"warn" + 0.058*"anti" + 0.039*"protest" + 0.017*"protesters" + '
  '0.015*"war" + 0.008*"travel" + 0.008*"rally" + 0.007*"terror" + '
  '0.006*"threat" + 0.006*"arrest"'),
 (3,
  '0.160*"pm" + 0.023*"troop" + 0.015*"north" + 0.014*"palestinian" + '
  '0.013*"west" + 0.012*"decision" + 0.010*"head" + 0.010*"hear" + '
  '0.009*"farmers" + 0.008*"clash"'),
 (4,
  '0.091*"health" + 0.074*"minister" + 0.064*"fund" + 0.063*"meet" + '
  '0.021*"troop" + 0.016*"service" + 0.008*"boost" + 0.007*"hospital" + '
  '0.007*"discuss" + 0.007*"group"'),
 (5,
  '0.097*"lead" + 0.046*"job" + 0.027*"tell" + 0.012*"shoo

In [418]:
keyword = 'china'
year = trend2003

for i in range(len(year)):
    if keyword in year.lemmatize.iloc[i]:
        print(year['headline_text'].iloc[i])

In [ ]:
# sklear

In [22]:
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');

In [23]:
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=10, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [24]:
model.components_[0].argsort()[:-20 - 1:-1]

array([ 770, 4199,  262,  286, 4008, 3972, 2483,  340, 1778, 1794, 1364,
       3503, 4920, 1622, 3006,  795, 1667, 3738, 2486, 2738], dtype=int64)

In [25]:
model.components_[0][:-20 - 1:-1]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.98983684e-05,
       0.00000000e+00, 2.90366825e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.49547581e-05, 0.00000000e+00,
       0.00000000e+00, 1.09998539e-02, 0.00000000e+00, 0.00000000e+00,
       1.02656925e-03, 5.62309427e-05, 2.86505275e-06, 3.37057920e-03])

In [26]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    vocabulary = vectorizer.vocabulary_
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [27]:
#get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,charge,plan,govt,face,win,continue,car,warn,soldier,fund
1,stab,development,nsw,trial,award,death,woman,war,claim,boost
2,assault,consider,urge,death,top,rise,hospital,travel,report,seek
3,attempt,reject,qld,water,lead,fight,accident,public,shoot,water
4,shoot,water,consider,tough,tour,investigation,fatal,water,baghdad,service
5,sex,house,feed,pair,title,protest,investigate,downer,iraqi,health
6,lay,group,accuse,ban,england,strike,stab,sars,troop,concern
7,bail,management,local,accuse,stage,house,house,power,blast,rain
8,fraud,protest,reject,restrictions,race,debate,death,threat,reject,indigenous
9,front,power,tas,future,final,clean,shoot,nsw,bomb,air
